In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
baseurl = 'http://elections.interieur.gouv.fr/presidentielle-2017/'

In [3]:
response = requests.get(baseurl)

In [4]:
doc = BeautifulSoup(response.text, 'html.parser')

In [5]:
links = doc.select('#carte_france area')
len(links)

358

In [6]:
url_list = []
for link in links:
    href = link.get('href')
    if href:
        url_list.append((href, link.get('title')))

In [7]:
url_list = list(set(url_list))

In [8]:
len(url_list)

107

In [9]:
targetCols = ['id', 'title', 'Abstention',
 'Nom',
 'Prenom',
 'Score',
 'Nom_1',
 'Prenom_1',
 'Score_1',
 'Statut']

usecols = ['id', 'title', 'Abstention',
 'Candidat1', 'Voix', '% Inscrits', '% Exprimés',
 'Candidat2', 'Voix2', '% Inscrits2', '% Exprimés2', 'Statut']

In [13]:
df = pd.DataFrame(columns=usecols)

for url in url_list:
    u = url[0]
    print(url[1], '…')
    absoluteURL = baseurl + u.replace('./', '')
    response = requests.get(absoluteURL)
    docDpt = BeautifulSoup(response.text, 'html.parser')
    dpt_nom = re.match('(.*) \(', url[1]).group(1)
    dpt_nom_check = docDpt.select('.pub-fil-ariane a')
    partiel = re.match('.* - (.*)', url[1]).group(1)

    if len(dpt_nom_check) > 2:
        print(dpt_nom, 'vs.', dpt_nom_check[2].string, url)
    
    no_match = re.match('.*\((.*?)\)', url[1])
    dpt_no = no_match.group(1)
    
    result_tables = docDpt.select('.tableau-resultats-listes-ER')
    if len(result_tables) > 1:
        rows = result_tables[0].select('tr')
        result_tables = docDpt.select('.tableau-resultats-listes-ER')
        rows = result_tables[0].select('tr')
        print(len(rows), 'rows found in', dpt_nom, ' – ', dpt_no)
        cleanRows = [dpt_no, dpt_nom]

        # 1) abstention
        abstention_tables = docDpt.select('.tableau-mentions')
        td_list = abstention_tables[0].select('td')
        for td in td_list:
            if td.string == 'Abstentions':
                #print(td, td.find_next_siblings()[1].string)
                abstentions = td.find_next_siblings()[1].string.strip()
                cleanRows.append(abstentions)

        # 2) candidats
        for row in rows[1:]:
            fields = row.select('td')
            fields_str = [f.string.strip() for f in fields]
            cleanRows.extend(fields_str)

        cleanRows.append(partiel)
        df = df.append(pd.DataFrame([(cleanRows)], columns=usecols))

Alpes-de-Haute-Provence (04) - résultats complets …
Alpes-de-Haute-Provence vs. Alpes-de-Haute-Provence (04) ('093/004/index.html', 'Alpes-de-Haute-Provence (04) - résultats complets')
3 rows found in Alpes-de-Haute-Provence  –  04
Essonne (91) - résultats complets …
Essonne vs. Essonne (91) ('011/091/index.html', 'Essonne (91) - résultats complets')
3 rows found in Essonne  –  91
Hérault (34) - résultats complets …
Hérault vs. Hérault (34) ('076/034/index.html', 'Hérault (34) - résultats complets')
3 rows found in Hérault  –  34
Jura (39) - résultats complets …
Jura vs. Jura (39) ('027/039/index.html', 'Jura (39) - résultats complets')
3 rows found in Jura  –  39
Nouvelle-Calédonie (988) - résultats complets …
3 rows found in Nouvelle-Calédonie  –  988
Saint-Pierre-et-Miquelon (975) - résultats complets …
3 rows found in Saint-Pierre-et-Miquelon  –  975
Lot (46) - résultats complets …
Lot vs. Lot (46) ('076/046/index.html', 'Lot (46) - résultats complets')
3 rows found in Lot  –  46
P

### NB: 79 dpts -> 86 dpts à 23h30 -> 96 à 23h56 -> 103 -> 104 à 0h34 -> 106

In [14]:
df.shape

(106, 12)

In [15]:
def getNom(value):
    if value.find('MACRON') > 0:
        return 'Macron'
    elif value.find('PEN') > 0:
        return 'Le Pen'
    else:
        return False

def getPrenom(value):
    if value.find('MACRON') > 0:
        return 'Emmanuel'
    elif value.find('PEN') > 0:
        return 'Marine'

In [16]:
def cleanNumbers(value):
    return value.replace(' ', '').replace(',', '.')

In [17]:
df_backup = df.copy()

In [18]:
df.head()

,id,title,Abstention,Candidat1,Voix,% Inscrits,% Exprimés,Candidat2,Voix2,% Inscrits2,% Exprimés2,Statut
0,04,Alpes-de-Haute-Provence,"23,13",M. Emmanuel MACRON,48 993,"38,74","58,46",Mme Marine LE PEN,34 816,"27,53","41,54",résultats complets
0,91,Essonne,"25,08",M. Emmanuel MACRON,382 650,"48,12","72,18",Mme Marine LE PEN,147 509,"18,55","27,82",résultats complets
0,34,Hérault,"24,70",M. Emmanuel MACRON,312 481,"39,05","59,22",Mme Marine LE PEN,215 174,"26,89","40,78",résultats complets
0,39,Jura,"21,58",M. Emmanuel MACRON,79 268,"41,65","61,37",Mme Marine LE PEN,49 887,"26,21","38,63",résultats complets
0,988,Nouvelle-Calédonie,"47,19",M. Emmanuel MACRON,47 902,"25,30","52,57",Mme Marine LE PEN,43 217,"22,82","47,43",résultats complets


# 79 -> 86 dpts à 23h30

In [19]:
df['Nom'] = df['Candidat1'].apply(getNom)
df['Prenom'] = df['Candidat1'].apply(getPrenom)

In [20]:
df['Nom_1'] = df['Candidat2'].apply(getNom)
df['Prenom_1'] = df['Candidat2'].apply(getPrenom)

In [21]:
for col in ['Abstention', 'Voix', '% Inscrits',
       '% Exprimés', 'Voix2', '% Inscrits2', '% Exprimés2']:
    df[col] = df[col].apply(cleanNumbers)

In [22]:
df['Score'] = df['% Exprimés']
df['Score_1']  = df['% Exprimés2']

In [23]:
df['Candidat1'].apply(getNom).value_counts()

Macron    104
Le Pen      2
Name: Candidat1, dtype: int64

In [24]:
def toInt(value):
    if value not in ('2A', '2B'):
        return int(value)
    else:
        return value
df['id'] = df['id'].apply(toInt)

In [25]:
df_noms = pd.read_csv('/Users/paul/Sites/elections_france/d3-composite-projections/data-t1.csv', usecols=['id', 'title'])

In [26]:
df_noms['id'] = df_noms['id'].apply(toInt)

In [27]:
df_noms[df_noms['title'].str.contains('Bourgogne')]

,id,title


In [28]:
dfm = df.merge(df_noms, how='outer', on='title')
del dfm['id_x']
dfm['id'] = dfm['id_y']

In [29]:
dfm

,title,Abstention,Candidat1,Voix,% Inscrits,% Exprimés,Candidat2,Voix2,% Inscrits2,% Exprimés2,Statut,Nom,Prenom,Nom_1,Prenom_1,Score,Score_1,id_y,id
0,Alpes-de-Haute-Provence,23.13,M. Emmanuel MACRON,48993,38.74,58.46,Mme Marine LE PEN,34816,27.53,41.54,résultats complets,Macron,Emmanuel,Le Pen,Marine,58.46,41.54,4,4
1,Essonne,25.08,M. Emmanuel MACRON,382650,48.12,72.18,Mme Marine LE PEN,147509,18.55,27.82,résultats complets,Macron,Emmanuel,Le Pen,Marine,72.18,27.82,91,91
2,Hérault,24.70,M. Emmanuel MACRON,312481,39.05,59.22,Mme Marine LE PEN,215174,26.89,40.78,résultats complets,Macron,Emmanuel,Le Pen,Marine,59.22,40.78,34,34
3,Jura,21.58,M. Emmanuel MACRON,79268,41.65,61.37,Mme Marine LE PEN,49887,26.21,38.63,résultats complets,Macron,Emmanuel,Le Pen,Marine,61.37,38.63,39,39
4,Nouvelle-Calédonie,47.19,M. Emmanuel MACRON,47902,25.30,52.57,Mme Marine LE PEN,43217,22.82,47.43,résultats complets,Macron,Emmanuel,Le Pen,Marine,52.57,47.43,NaN,NaN
5,Saint-Pierre-et-Miquelon,44.01,M. Emmanuel MACRON,1467,29.56,63.29,Mme Marine LE PEN,851,17.15,36.71,résultats complets,Macron,Emmanuel,Le Pen,Marine,63.29,36.71,NaN,NaN
6,Lot,20.03,M. Emmanuel MACRON,66937,49.12,72.18,Mme Marine LE PEN,25802,18.94,27.82,résultats complets,Macron,Emmanuel,Le Pen,Marine,72.18,27.82,46,46
7,Pyrénées-Atlantiques,21.96,M. Emmanuel MACRON,253617,50.49,74.81,Mme Marine LE PEN,85377,17.00,25.19,résultats complets,Macron,Emmanuel,Le Pen,Marine,74.81,25.19,64,64
8,Somme,22.13,M. Emmanuel MACRON,153242,37.47,54.19,Mme Marine LE PEN,129545,31.68,45.81,résultats complets,Macron,Emmanuel,Le Pen,Marine,54.19,45.81,80,80
9,Ardennes,25.67,M. Emmanuel MACRON,64423,33.15,50.73,Mme Marine LE PEN,62571,32.20,49.27,résultats complets,Macron,Emmanuel,Le Pen,Marine,50.73,49.27,8,8


In [30]:
dfm[targetCols].to_csv('/Users/paul/Sites/elections_france/d3-composite-projections/data-t2.csv', index=False)

In [31]:
print(*dfm[dfm['Abstention'] != dfm['Abstention']]['title'].values, sep=', ')

In [32]:
df.shape

(106, 18)

In [33]:
df['Statut'].value_counts()

résultats complets    106
Name: Statut, dtype: int64